In [6]:
import pandas as pd
import qgrid

import os

from IPython.core.display import Javascript
from IPython.display import display

from highcharts import Highstock

In [17]:
path = '/Users/martin/Google Drive/data/screener/'
files = os.listdir(path)
    
dff = pd.DataFrame()
dff['Instrument'] = files

qgw = qgrid.show_grid(dff)

In [18]:
line_no = 0

def handle_selection_changed(event, widget):
    global line_no
    line_no = event['new'][0]
    # todo: get filtered and sorted df
    build_chart()

qgw.on('selection_changed', handle_selection_changed)

In [21]:
def build_chart():
    filename = dff.iloc[line_no,]['Instrument']
    filepath = '/Users/martin/Google Drive/data/screener/'
    df = pd.read_csv(filepath + filename)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df['date'] = (df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')
    df = df.drop(columns=['volume'])

    chart = Highstock()
    data = df[['date','open','high','low','close']].values.tolist()
    chart.add_data_set(data, series_type='ohlc', name=filename.replace('.csv', ''))
    # todo: add moving average

    options = {
        'chart': {
            'animation': False,
            'height': 600,
            'width': 950
        },
        'rangeSelector': {
            'selected': 4
        },
        'title': {
            'text': filename.replace('.csv', '')
        },
        'plotOptions': {
            'series': {
                'animation': False
            }
        },
        'yAxis': {
            'type': 'logarithmic'
        }
    }
    chart.set_dict_options(options)
    chart.save_file('./website/index')

In [20]:
qgw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…